#### 1. Importing Required libraries

In [1]:
from dotenv import load_dotenv

load_dotenv()
import json
import os

from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import DeepLake

c:\Users\krish\miniconda3\envs\usercom\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


#### 2. Loading the Hyd data

In [3]:
#loader = TextLoader('data/articles_content.txt')
loader = CSVLoader('nyc_places.csv')

In [4]:
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=20
)

docs = text_splitter.split_documents(documents)

In [5]:
embeddings = VertexAIEmbeddings()
dataset_path="hub://travana_db/nyc_embedding"

db = DeepLake(embedding=embeddings, dataset_path=dataset_path, overwrite=True)
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating embedding data: 100%|██████████| 13/13 [00:08<00:00,  1.60it/s]
-

Dataset(path='hub://travana_db/nyc_embedding', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (171, 768)  float32   None   
    id        text      (171, 1)     str     None   
 metadata     json      (171, 1)     str     None   
   text       text      (171, 1)     str     None   


['ca99c5d2-771f-11ee-b082-04421acf6dbe',
 'ca99c5d3-771f-11ee-b8f9-04421acf6dbe',
 'ca99c5d4-771f-11ee-9aea-04421acf6dbe',
 'ca99c5d5-771f-11ee-8297-04421acf6dbe',
 'ca99c5d6-771f-11ee-9092-04421acf6dbe',
 'ca99c5d7-771f-11ee-bdca-04421acf6dbe',
 'ca99c5d8-771f-11ee-a0b8-04421acf6dbe',
 'ca99c5d9-771f-11ee-9f19-04421acf6dbe',
 'ca99c5da-771f-11ee-99ea-04421acf6dbe',
 'ca99c5db-771f-11ee-bac4-04421acf6dbe',
 'ca99c5dc-771f-11ee-b229-04421acf6dbe',
 'ca99c5dd-771f-11ee-b51c-04421acf6dbe',
 'ca99c5de-771f-11ee-9369-04421acf6dbe',
 'ca99c5df-771f-11ee-bdf0-04421acf6dbe',
 'ca99c5e0-771f-11ee-9e80-04421acf6dbe',
 'ca99c5e1-771f-11ee-96ea-04421acf6dbe',
 'ca99c5e2-771f-11ee-a556-04421acf6dbe',
 'ca99c5e3-771f-11ee-b910-04421acf6dbe',
 'ca99c5e4-771f-11ee-ba1e-04421acf6dbe',
 'ca99c5e5-771f-11ee-8dfa-04421acf6dbe',
 'ca99c5e6-771f-11ee-a04c-04421acf6dbe',
 'ca99c5e7-771f-11ee-88e9-04421acf6dbe',
 'ca99c5e8-771f-11ee-9209-04421acf6dbe',
 'ca99c5e9-771f-11ee-88f1-04421acf6dbe',
 'ca99c5ea-771f-

In [6]:
db_loaded = DeepLake(
    dataset_path=dataset_path, 
    embedding=embeddings, 
    token=os.getenv("ACTIVELOOP_TOKEN"),
    read_only=True
)

Deep Lake Dataset in hub://travana_db/nyc_embedding already exists, loading from the storage


In [7]:
retriever = db_loaded.as_retriever()

In [19]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
You can assume the question about the most recent state of the union address.

Chat History:
{chat_history}

Follow Up Input: {question}

Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [26]:
template = """
            You are an expert intelligent and creative AI itinerary planner with extensive knowledge of places worldwide. Your goal is to plan an optimized itinerary for the user based on their specific interests and preferences, geographical proximity, and efficient routes to minimize travel time. To achieve this, follow these instructions:

            1. Suggest three activities along with two restaurant per day. Each activity should include the name of the place, a brief description, estimated cost, and time to visit the place.

            2. Generate a well-structured itinerary including day-to-day activities, timings to visit each location, and estimated costs for the user's reference.

            3. Take into account factors such as geographical proximity between destinations, transportation options, and other logistical considerations when planning the route.
            
            By following these guidelines, you will create a comprehensive and optimized itinerary that meets the user's expectations while ensuring minimal travel time.
          
            Use this context while generating the itinerary. Consider the best time to visit and average duration to spend at each place.
            
            ==========
            {context}
            ==========

            Question: {question}

            Structure the itinerary as follows:
            {{"Name":"name of the trip", "description":"description of the entire trip", "budget":"budget of the entire thing", "data": [{{"day":1, "day_description":"Description based on the entire day's places. in a couple of words, for example: `Urban Exploration`, `Historical Exploration`, `Spiritual Tour`, `Adventurous Journey`, `Dayout in a beach`, `Wildlife Safari`, `Artistic Getaway`, `Romantic Getaway`, `Desert Safari`, `Island Hopping Adventure`...",  "places":[{{"name":"Place Name", "description":"Place Description in two lines","time_to_visit": "best time to visit this place, for example: 9:00 to 10:00",  "average_duration":"average duration to spend at this place", "budget":"cost"}}, {{"name":"Place Name 2", "description":"Place Description 2 in two lines","time_to_visit": "best time to visit this place, for example 10:30 - 11:30", "average_duration":"average duration to spend at this place", "budget":"cost"}}]}}, {{"day":2, "day_description": "Description based on the entire day's places", "places":[{{"name":"Place Name","description":"Place Description in two lines","time_to_visit": "best time to visit this place", "average_duration":"average duration to spend at this place", "budget":"cost"}}, {{"name":"Place Name 2", "description":"Place Description 2 in two lines","time_to_visit": "best time to visit this place", "average_duration":"average duration to spend at this place","budget":"cost"}}]}}]}}
            Note: Do not include any extra information outside this structure.

            Answer in JSON format:"""

In [27]:
QA_PROMPT = PromptTemplate(template=template, input_variables=[
                           "context", "question"])

In [28]:
llm = ChatVertexAI(model_name='chat-bison', temperature=0.75, max_output_tokens=2004)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_docs=True
)

model = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    get_chat_history=lambda h: h,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT}
)

In [29]:
default_question = """Generate 3 day itinerary for a trip to New York City, USA from November 1st to November 3rd with $400 budget.
My day starts from 9 am and ends at 8 pm. 
Consider budget, best time to visit and average duration to spend at each place.
Include estimated cost and timings to visit for each activity and restaurant"""


In [30]:
#query = Prompt.ask("Your Question: ",  default=default_question)
result = model({"question": default_question})
print(result['answer'])

 ```JSON
{
  "name": "New York City Trip",
  "description": "A 3-day itinerary for a trip to New York City, USA.",
  "budget": "$400",
  "data": [
    {
      "day": 1,
      "day_description": "Urban Exploration",
      "places": [
        {
          "name": "Times Square",
          "description": "A bustling crossroads of Broadway and Seventh Avenue, known for its bright lights, Broadway shows, and giant electronic billboards.",
          "time_to_visit": "9:00 AM - 10:00 AM",
          "average_duration": "2 hours",
          "budget": "$0"
        },
        {
          "name": "Central Park",
          "description": "An 843-acre park in the heart of Manhattan, featuring lakes, forests, meadows, and various attractions such as the Central Park Zoo and the Wollman Rink.",
          "time_to_visit": "10:30 AM - 12:30 PM",
          "average_duration": "2 hours",
          "budget": "$0"
        },
        {
          "name": "The Metropolitan Museum of Art",
          "description

In [ ]:
result = model({"question": "Change the day 3 itinerary only to include restaurants"})
print(result['answer'])